### Подсчет у таблицы (Merged and cleaned) mdm фичей 
Подсчет у таблицы (Merged and cleaned) mdm фичей 
Входные данные data/interm/merge_cleaned.pkl из merge_and_clean.ipynb  
 
  
Подсчитывает по данным федресурса аналогичные фичи, используемые для обучения моделей в target class

|    | feature                                | Наименование                                                                         |
|---:|:---------------------------------------|:-------------------------------------------------------------------------------------|
|  0 | no_contracts_total                     | Количество договоров (всех закрытых и действующих)                                   |
|  1 | no_contracts_active                    | Количество активных (действующих) договоров                                          |
|  2 | no_transport_total                     | Количество договоров только с транспортными средствами (всех закрытых и действующих) |
|  3 | no_transp_active                       | Количество активных (действующих) договоров только с транспортными средствами        |
|  4 | no_contracts_closed                    | Количество закрытых договоров                                                        |
|  5 | reason_завершение                      | Количество закрытых договоров по причине завершения договора                         |
|  6 | reason_замена/перенайм/переуступка     | Количество закрытых договоров по причине замена/перенайм/переуступка                 |
|  7 | reason_не указана                      | Количество закрытых договоров по неуказанной причине                                 |
|  8 | reason_определена цессия               | Количество закрытых договоров по причине цессии                                      |
|  9 | reason_отмена                          | Количество закрытых договоров по причине отмены                                      |
| 10 | reason_расторгнут                      | Количество закрытых договоров по причине расторжения                                 |
| 11 | reason_страховой случай                | Количество закрытых договоров по причине страхового случая                           |
| 12 | reason_страховой случай (гибель/тотал) | Количество закрытых договоров по причине страхового случая (гибель/тотал)            |
| 13 | reason_гибель/тотал                    | Количество закрытых договоров по причине гибель/тотал                                |
| 14 | bad_reasons_closed                     | Количество договоров c признаками плохого завершения                                 |
| 15 | good_reasons_closed                    | Количество договоров c признаками хорошего завершения                                |
| 16 | not_determined_reasons_closed          | Количество договоров с неопределенным завершением                                    |
| 17 | mean_bad_reasons                       | Среднее количество закрытых договоров с отрицательным исходом                        |
| 18 | mean_good_reasons                      | Среднее количество закрытых договоров с положительным исходом                        |
| 19 | mean_not_determined_reasons            | Среднее количество закрытых договоров с неопределенным исходом                       |
| 20 | no_creditors                           | Количество различных лизингодателей                                                  |
| 21 | duration_max                           | Максимальная длительность закрытых договоров                                         |
| 22 | duration_min                           | Минимальная длительность закрытых договоров                                          |
| 23 | duration_mean                          | Средняя длительность закрытых договоров                                              |
| 24 | contract_date_delta_first              | Разница до даты заключения первого договора                                          |
| 25 | contract_date_delta_last               | Разница до даты заключения последнего договора                                       |
| 26 | end_date_last                          | Разница до даты самого последнего завершения договора                                |
| 27 | end_date_first                         | Разница до даты самого первого завершения договора                                   |
 
  
Все фичи рассчитываются на последние данные в разрезе new_lessee_inn  
Записываем в data/interm/features_mdm.pkl файл для дальнейшего анализа  


In [1]:
import pandas as pd

In [2]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb
%run ../preprocessing/preprocessing_target.ipynb
%run ../ML_functions.ipynb
%run ../auxiliary/plots.ipynb

In [3]:
# сюда таблицы будем кидать, для отчетов
name = pre + "data/interm/auxilary/feature_engineering_mdm.xlsx"
writer = ExcelWriter(name)

In [4]:
name = pre + "data/interm/merge_cleaned.pkl"
df_mdm = pd.read_pickle(name, compression={'method':'zip'})

print("Размер считанных данных фед ресурса mdm- {}".format(df_mdm.shape))

Размер считанных данных фед ресурса mdm- (1443142, 28)


In [5]:
# df_mdm.head(1)

In [6]:
current_day = datetime.now()
current_day

datetime.datetime(2023, 10, 17, 16, 18, 23, 16590)

## 1. посчитаем, количество договоров на дату публикации

In [7]:
contacts_total = df_mdm.groupby('new_lessee_inn')['origin_by_message'].aggregate('nunique').to_frame()\
                    .reset_index().rename(columns={'origin_by_message':'no_contracts_total'})
print(len(contacts_total))
# contacts_total.head()

229912


## 2. посчитаем, количество договоров на дату публикации (только транспортных средств)

In [8]:
transport_only = df_mdm[(df_mdm['new_leasingsubject_code'].notnull()) & (df_mdm['new_leasingsubject_code'].str.startswith('0106'))]
contacts_transp_total = transport_only.groupby('new_lessee_inn')['origin_by_message'].aggregate('nunique').to_frame()\
                                        .reset_index().rename(columns={'origin_by_message':'no_transport_total'})

## 3. посчитаем количество активных договоров

для этого оставим последнюю запись по договам из mdm к нашим приложениям  и там посмотрим, завершен ли там договор или нет

In [9]:
last = df_mdm.sort_values(by=['new_lessee_inn', 'origin_by_message', 'new_kreditmessage_date']).drop_duplicates(subset=['new_lessee_inn', 'origin_by_message'], keep='last')

In [10]:
"""stop_primary - предварительно вычисленная дата окончания, там где явно задано окончание, то  его указываем, там где не указано, то по договору лизинга"""
mask = (last['cession_date'].notnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'cession_date']

mask = (last['new_terminationleasing_date'].notnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'new_terminationleasing_date']

mask = (last['stop_primary'].isnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'new_leasing_enddate']

mask = (last['stop_primary'].isnull())
mask_stop = (last['new_kreditmessage_type'] == 'StopFinancialLeaseContract')
last.loc[mask & mask_stop, 'stop_primary'] = last.loc[mask & mask_stop, 'new_kreditmessage_date']

# get_contract_by_number(df_mdm, 'Ак-СПб-0218/2463')	
# get_contract_by_number(last, 'Ак-СПб-0218/2463')	

In [11]:
"""
уберем те, уже закрытые на момент принятия решения, будем считать только активные договоры
"""
mask_closed = (last['stop_primary']<=current_day)
raw_active = last[~mask_closed].copy()

In [12]:
active = raw_active.groupby('new_lessee_inn')['origin_by_message'].aggregate('nunique').to_frame().reset_index().rename(columns={'origin_by_message':'no_contracts_active'})
# active.head()

### 4. Теперь считаем только активные по транспортным средствам

In [13]:
transport_only_active = raw_active[(raw_active['new_leasingsubject_code'].notnull()) & (raw_active['new_leasingsubject_code'].str.startswith('0106'))]
transp_active = transport_only_active.groupby('new_lessee_inn')['origin_by_message'].aggregate('nunique').to_frame()\
                            .reset_index().rename(columns={'origin_by_message':'no_transp_active'})
# transp_active.head()

### 5. Расчет фичей:
- no_contracts_closed -  количество завершенных договоров
- количества причин завершения у нашего лизингополучателя:

In [14]:
"""
уберем все активные на момент принятия решения, будем считать только закрытые договоры

"""
mask_closed = (last['stop_primary']<=current_day)

raw_closed = last[mask_closed].copy()

mask = (raw_closed['reason']=='')
raw_closed.loc[mask, 'reason'] = 'не указана'

In [15]:
closed = raw_closed.groupby('new_lessee_inn')['origin_by_message'].aggregate('nunique').to_frame().reset_index().rename(columns={'origin_by_message':'no_contracts_closed'})
# closed.head()

In [16]:
dummies = pd.get_dummies(raw_closed["reason"], prefix="reason")
col_reasons = dummies.columns.to_list()
raw_closed_reasons = raw_closed.join(dummies)
closed_reasons = raw_closed_reasons.groupby('new_lessee_inn')[col_reasons].aggregate('sum').reset_index()
# closed_reasons.head()

bad_reasons_closed - плохое завершение предыдущих договоров  
good_reasons_closed - хорошее завершение предыдущих договоров  
not_determined_reasons_closed - неопределенное завершение предыдущих договоров  

In [17]:
bad = ['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)']
good = ['reason_завершение']
not_determined = ['reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай']

In [18]:
closed_reasons['bad_reasons_closed'] = closed_reasons[bad].sum(axis=1)
closed_reasons['good_reasons_closed'] = closed_reasons[good].sum(axis=1)
closed_reasons['not_determined_reasons_closed'] = closed_reasons[not_determined].sum(axis=1)

# closed_reasons.head()

In [19]:
# небольшая проверка, что все закрытые договоры были разнесены на 3 подгруппы
closed_w_reasons = closed_reasons.copy()
closed_w_reasons = closed_w_reasons.fillna(0)

closed_w_reasons['closed'] = closed_w_reasons[['bad_reasons_closed',	'good_reasons_closed',	'not_determined_reasons_closed']].sum(axis=1)
closed_w_reasons = pd.merge(closed, closed_w_reasons , how = 'outer', on='new_lessee_inn')
Check_length(len(closed_w_reasons), len(closed))

tmp = check_fields(closed_w_reasons,'closed', 'no_contracts_closed' )

closed_w_reasons.drop(columns='closed', inplace=True)

In [20]:
mask = (closed_w_reasons['no_contracts_closed']==0)

closed_w_reasons.loc[mask, 'mean_bad_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_bad_reasons'] = closed_w_reasons.loc[~mask, 'bad_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

closed_w_reasons.loc[mask, 'mean_good_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_good_reasons'] = closed_w_reasons.loc[~mask, 'good_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

closed_w_reasons.loc[mask, 'mean_not_determined_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_not_determined_reasons'] = closed_w_reasons.loc[~mask, 'not_determined_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

# closed_w_reasons.head(1)

### 6. Длительность закрытых договоров

In [21]:
raw_closed['Длительность'] = raw_closed['stop_primary'] - raw_closed['new_leasingcontract_date']
mask = (raw_closed['Длительность']<pd.Timedelta(days=0))
raw_closed.loc[mask, 'Длительность'] = pd.Timedelta(days=0)

duration = pd.pivot_table(raw_closed, index='new_lessee_inn',  values='Длительность',  aggfunc={'min', 'mean', 'max'})\
    .rename(columns={'max':'duration_max', 'mean':'duration_mean', 'min':'duration_min'})

for col in duration.columns:
    if col == 'new_lessee_inn':
        continue
    duration[col] = duration[col].round('D')
duration.reset_index(inplace=True)
# duration.head(3)

### 7. Разница между текущей расчетной датой и датой первого и последнего договора в фед ресурсе

In [22]:
"""
у некоторых договоров дата договора не проставлена 
у таких договоров разница не имеет смысла, она будут исключены их расчета
"""
tmp = last[last['new_leasingcontract_date'].isnull()]
print("дата договора не проставлена у ", len(tmp))
# tmp.head(1)

дата договора не проставлена у  1560


In [23]:
"""
у некоторых договоров дата договора стоит неправильная, больше текущей даты, 
у некоторых договоров дата договора не проставлена 
у таких договоров разница не имеет смысла, она будут исключены их расчета
"""
last['contract_date_delta'] = current_day - last['new_leasingcontract_date']
tmp = last[last['contract_date_delta']>=pd.Timedelta(days=0)]
contract_timedelta = pd.pivot_table(tmp, index='new_lessee_inn',  values='contract_date_delta', aggfunc={'min', 'max'})\
    .rename(columns={'max':'contract_date_delta_first', 'min':'contract_date_delta_last'}).reset_index()

contract_timedelta['contract_date_delta_last'] = contract_timedelta['contract_date_delta_last'].round('D')
contract_timedelta['contract_date_delta_first'] = contract_timedelta['contract_date_delta_first'].round('D')
# contract_timedelta.head()

In [24]:
contract_timedelta.describe()

,contract_date_delta_first,contract_date_delta_last
count,229138,229138
mean,1261 days 02:26:47.492428176,886 days 10:39:22.628634256
std,620 days 16:48:36.323983336,598 days 06:39:36.649892320
min,42 days 00:00:00,23 days 00:00:00
25%,807 days 00:00:00,408 days 00:00:00
50%,1178 days 00:00:00,799 days 00:00:00
75%,1692 days 00:00:00,1225 days 00:00:00
max,20559 days 00:00:00,7782 days 00:00:00


### 8. Разница между текущей датой и датой закрытия договора в фед ресурсе

In [25]:
raw_closed['contract_date_delta'] = current_day - raw_closed['stop_primary']

end_date_timedelta = pd.pivot_table(raw_closed, index='new_lessee_inn',  values='contract_date_delta', aggfunc={'min', 'max'})\
    .rename(columns={'max':'end_date_first', 'min':'end_date_last'}).reset_index()

end_date_timedelta['end_date_last'] = end_date_timedelta['end_date_last'].round('D')
end_date_timedelta['end_date_first'] = end_date_timedelta['end_date_first'].round('D')
# end_date_timedelta.head()

In [26]:
#  0 у кого сегодня по плану завершение
end_date_timedelta.describe()

,end_date_first,end_date_last
count,150908,150908
mean,795 days 03:48:38.493386680,514 days 12:40:04.643888992
std,718 days 04:51:40.046431392,470 days 12:09:29.997705944
min,1 days 00:00:00,1 days 00:00:00
25%,327 days 00:00:00,140 days 00:00:00
50%,667 days 00:00:00,370 days 00:00:00
75%,1220 days 00:00:00,730 days 00:00:00
max,98905 days 00:00:00,3244 days 00:00:00


### 9. Количество различных лизингодателей

In [27]:
last['new_creditor_inn'] = last['new_creditor_inn'].fillna("")
creditors = last.groupby('new_lessee_inn')['new_creditor_inn'].aggregate('nunique').to_frame().reset_index().rename(columns={'new_creditor_inn':'no_creditors'})
# creditors.head()

In [28]:
creditors.describe()

,no_creditors
count,229912.000000
mean,1.478831
std,1.079071
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,27.000000


## Соединяем все фичи в один Dataframe и записываем в pkl файл

In [29]:
# contacts_total содержит полный перечень new_lessee_inn, поэтому берется за основу

tmp = pd.merge(contacts_total, contacts_transp_total , how = 'left', on='new_lessee_inn')

tmp = pd.merge(tmp, active, how = 'left', on='new_lessee_inn')
tmp = pd.merge(tmp, transp_active , how = 'left', on='new_lessee_inn')

tmp = pd.merge(tmp, closed_w_reasons , how = 'left', on='new_lessee_inn')
tmp = pd.merge(tmp, creditors , how = 'left', on='new_lessee_inn')

tmp = pd.merge(tmp, duration , how = 'left', on='new_lessee_inn')
tmp = pd.merge(tmp, contract_timedelta , how = 'left', on='new_lessee_inn')
features = pd.merge(tmp, end_date_timedelta , how = 'left', on='new_lessee_inn')


Check_length(len(contacts_total), len(features))

list_of_features = ['no_contracts_active', 'no_transp_active', 'no_contracts_total', 
                    'no_transport_total', 'no_contracts_closed', 'bad_reasons_closed', 
                    'good_reasons_closed', 'not_determined_reasons_closed', 
                    'mean_bad_reasons','mean_good_reasons', 'mean_not_determined_reasons', 
                    'no_creditors']

for col in list_of_features:
    features[col] = features[col].fillna(0)

for col in col_reasons:
    features[col] = features[col].fillna(0)

# features.head(1)

## Статистика по фичам

In [30]:
describe = features.describe().T

null_stat = pd.DataFrame(round(features.isnull().sum()/len(features)*100, 2), 
                        columns=['Процент null значений'])
stat = describe.join(null_stat).reset_index().rename(columns={'index':'feature'})

tmp = pd.DataFrame(allFeaturesProperties).T
stat = pd.merge(stat, tmp[['short_name']], left_on='feature', right_index=True, how='left').rename(columns = {'short_name':'Наименование'})

for row in ['duration_max', 'duration_mean', 'duration_min',
            'contract_date_delta_first', 'contract_date_delta_last',
            'end_date_first', 'end_date_last' ]:
    mask = (stat['feature']==row)
    stat.loc[mask, 'min'] = stat.loc[mask, 'min'].apply(strfdelta, fmt='{D}days')
    stat.loc[mask,'max'] = stat.loc[mask,'max'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'mean'] = stat.loc[mask,'mean'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'std'] = stat.loc[mask,'std'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'25%'] = stat.loc[mask,'25%'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'50%'] = stat.loc[mask,'50%'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'75%'] = stat.loc[mask,'75%'].apply(strfdelta, fmt='{Y} years  {D}days')


stat.to_excel(writer, sheet_name='feature_stat')

stat

,feature,count,mean,std,min,25%,50%,75%,max,Процент null значений,Наименование
0,no_contracts_total,229912.0,4.898335,23.577112,1.0,1.0,2.0,4.0,7249.0,0.00,Количество договоров (всех закрытых и действую...
1,no_transport_total,229912.0,3.71834,21.245554,0.0,1.0,1.0,3.0,7168.0,0.00,Количество договоров только с транспортными ср...
2,no_contracts_active,229912.0,2.470837,12.011449,0.0,0.0,1.0,2.0,1767.0,0.00,Количество активных (действующих) договоров
3,no_transp_active,229912.0,1.804973,10.124078,0.0,0.0,0.0,1.0,1737.0,0.00,Количество активных (действующих) договоров то...
4,no_contracts_closed,229912.0,2.427498,15.054855,0.0,0.0,1.0,2.0,5482.0,0.00,Количество закрытых договоров
5,reason_гибель/тотал,229912.0,0.001144,0.038608,0.0,0.0,0.0,0.0,6.0,0.00,Количество закрытых договоров по причине гибел...
6,reason_завершение,229912.0,0.89603,4.945144,0.0,0.0,0.0,1.0,1391.0,0.00,Количество закрытых договоров по причине завер...
7,reason_замена/перенайм/переуступка,229912.0,0.001831,0.122998,0.0,0.0,0.0,0.0,29.0,0.00,Количество закрытых договоров по причине замен...
8,reason_не указана,229912.0,1.266063,11.684928,0.0,0.0,0.0,1.0,4861.0,0.00,Количество закрытых договоров по неуказанной п...
9,reason_определена цессия,229912.0,0.050376,0.897364,0.0,0.0,0.0,0.0,233.0,0.00,Количество закрытых договоров по причине цессии


In [31]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save()

In [32]:
features.to_pickle(pre+"data/interm/features_mdm.pkl", compression={'method':'zip'})